In [1]:
##要执行的我会在代码段前注释“第i步”
#第一步
import numpy
import pandas as pd
import datetime
import json
import os
import xlrd# 安装xlrd == 1.2.0 版本太高会出错
import numpy as np
import matplotlib.pyplot as plt
import copy
from skyfield.api import load, EarthSatellite
from skyfield.api import wgs84
from matplotlib.pyplot import MultipleLocator
import matplotlib.ticker as mtick
import matplotlib.dates as mdates

## No need to run

In [2]:
def get_allfile(path):  # 获取所有文件
    all_file = []
    for f in os.listdir(path):  #listdir返回文件中所有目录
        f_name = os.path.join(path, f)
        all_file.append(f_name)
    return all_file

In [3]:
#第三步
#从TLE中提取相关字段到tle列表中
tle=[]
key_wanted=["NORAD_CAT_ID","OBJECT_NAME","EPOCH","INCLINATION","TLE_LINE1","TLE_LINE2","SEMIMAJOR_AXIS"]
tickets = r"../../Dataset/starlink/"
all_file=get_allfile(tickets)  #tickets要获取文件夹名
for i in range(len(all_file)):
    filename=r"%s"%all_file[i]
    #print(filename)
    with open(filename) as f:
        temp=json.load(f)
        for item in temp:
            item["NORAD_CAT_ID"]=int(item["NORAD_CAT_ID"])
            item["SEMIMAJOR_AXIS"] = float(item["SEMIMAJOR_AXIS"])
            item["EPOCH"]=datetime.datetime.strptime('%s.%s'%(item["EPOCH"],item["EPOCH_MICROSECONDS"]),"%Y-%m-%d %H:%M:%S.%f")
            temp2=dict((key,value) for key, value in item.items() if key in key_wanted)
            tle.append(temp2)
df=pd.DataFrame(tle)

#提取到tle后按“NORAD_CAT_ID”分组，并按EPOCH的逆序排序
group = df.groupby("NORAD_CAT_ID")
starlink_dict = {}
for dfGroupBy in group:
    df_dict_key = dfGroupBy[0]
    df_dict_value = pd.DataFrame(dfGroupBy[1])
    starlink_dict[df_dict_key] = df_dict_value.sort_values(by='EPOCH',ascending=False)
#print(starlink_dict['STARLINK-3282'])

In [4]:
maneuver = {}
for i in range(len(starlink_dict.keys())):#len(starlink_dict.keys())
    print(i)
    starlink_maneuver = {}
    starlink_dataframe = starlink_dict[list(starlink_dict.keys())[i]]
    starlink_dataframe = starlink_dataframe[starlink_dataframe['EPOCH'] > datetime.datetime(2022,4,1)]
    if(len(starlink_dataframe) == 0):continue
    axis_1 = float(starlink_dataframe.iloc[0,6])
    axis_2 = float(starlink_dataframe.iloc[len(starlink_dataframe) - 1,6])
    if(abs(axis_1 - axis_2) >= 10):continue
    for j in range(len(starlink_dataframe)):
        axis_sum = 0
        num_sum = 0
        for k in range(5):
            if(j + k >= len(starlink_dataframe)):break
            num_sum = num_sum + 1
            axis_sum = axis_sum + float(starlink_dataframe.iloc[j + k,6])
        for seq in range(5):
            if(j + seq >= len(starlink_dataframe)):break
            new_axis_sum = axis_sum - float(starlink_dataframe.iloc[j + seq,6])
            if(num_sum - 1 == 0):continue
            axis_avg = new_axis_sum / (num_sum - 1)
            if(abs(float(starlink_dataframe.iloc[j + seq,6]) - axis_avg) >= 1):
                starlink_maneuver[starlink_dataframe.iloc[j + seq,2]] = float(starlink_dataframe.iloc[j + seq,6])
    maneuver[list(starlink_dict.keys())[i]] = starlink_maneuver

    

0
1
2
3


ValueError: could not convert string to float: '2 44238  53.0066 321.3434 0000458  21.3132 338.7890 15.39630065174752'

In [ ]:
def ismaneuver(key,start_time,stop_time):
    df = starlink_dict[key]
    df = df[(start_time <= df['EPOCH']) & (df['EPOCH'] <= stop_time)]
    df_axis_list = []
    for i in range(len(df)):
        df_axis_list.append(df.iloc[i,6])
    df_axis_max = max(df_axis_list)
    #print(df_axis_max)
    df_axis_min = min(df_axis_list)
    #print(df_axis_min)
    if((df_axis_max - df_axis_min) < 1):return False
    else:return True

In [ ]:
start_end={}
for key in maneuver.keys():
    if(maneuver[key] == {}):continue
    tmp_man=list(sorted(maneuver[key].keys()))
    starlink_df = starlink_dict[key]
    starlink_df = starlink_df[starlink_df['EPOCH'] > datetime.datetime(2022,4,1)]
    #print(np.array(starlink_df["SEMIMAJOR_AXIS"]))
    semi=np.median(np.array(starlink_df["SEMIMAJOR_AXIS"]))
    start=[]
    end=[]
    for i in range(len(tmp_man)):
        item=tmp_man[i]
        tmp_start=starlink_df[starlink_df['EPOCH'] < item]
        tmp_start=tmp_start[abs(tmp_start['SEMIMAJOR_AXIS']-semi)<0.2]
        t=tmp_start['EPOCH'].max()
        if t not in start:
            if len(start)!=0:
                if abs((start[-1]-t).total_seconds())>3600*12:
                    start.append(t)
            else:
                start.append(t)
        
        tmp_end=starlink_df[starlink_df['EPOCH'] > item]
        tmp_end=tmp_end[abs(tmp_end['SEMIMAJOR_AXIS']-semi)<0.2]
        t=tmp_end['EPOCH'].min()
        if t not in end :
            if len(end)!=0:
                if abs((end[-1]-t).total_seconds())>3600*12:
                    end.append(t)
                else:
                    end[-1]=t
            else:
                end.append(t)
    if(key == 48014):
        print(start)
        print(end)
    start = sorted(start)
    end = sorted(end,reverse=True)
    del_list1 = []
    flag1 = 0
    for l1 in range(1,len(start)):
        if((start[l1] - start[flag1]) < datetime.timedelta(days=20)):
            del_list1.append(start[l1])
        else:
            flag1 = l1
    for l1 in range(len(del_list1)):
        start.remove(del_list1[l1])
    del_list2 = []
    flag2 = 0
    for l2 in range(1,len(end)):
        if((end[flag2] - end[l2]) < datetime.timedelta(days=20)):
            del_list2.append(end[l2])
        else:
            flag2 = l2
    for l2 in range(len(del_list2)):
        end.remove(del_list2[l2])
    end.reverse()
    if(key == 48014):
        print(start)
        print(end)
    result = []
    i#f(len(start) != len(end)):
    for x in range(len(start)):
        time_min = datetime.timedelta(days = 999)
        time_ind = 'a'
        for y in range(len(end)):
            if(end[y] <= start[x]):continue
            else:
                if((end[y] - start[x]) < time_min):
                    #print(datetime.datetime.strptime(str(start[x]),"%Y-%m-%d %H:%M:%S.%f"))
                    if(ismaneuver(key,datetime.datetime.strptime(str(start[x]),"%Y-%m-%d %H:%M:%S.%f"),datetime.datetime.strptime(str(end[y]),"%Y-%m-%d %H:%M:%S.%f"))):
                        time_min = end[y] - start[x]
                        time_ind = y
        if(time_ind == 'a'):continue
        result.append([start[x],end[time_ind]])
    # else:
    #     for x in range(len(start)):
    #         result.append([start[x],end[x]])
    start_end[key]=result

In [ ]:
task_maneuver = []
for i in range(len(maneuver.keys())):
    if(maneuver[list(maneuver.keys())[i]] == {}):continue
    starlink_dataframe = starlink_dict[list(maneuver.keys())[i]]
    starlink_dataframe = starlink_dataframe[starlink_dataframe['EPOCH'] > datetime.datetime(2022,4,1)]
    if(len(starlink_dataframe) == 0):continue
    axis_1 = float(starlink_dataframe.iloc[0,6])
    axis_2 = float(starlink_dataframe.iloc[len(starlink_dataframe) - 1,6])
    if(abs(axis_1 - axis_2) >= 10):continue
    time_num = 0
    for j in range(len(starlink_dataframe)):
        if(float(starlink_dataframe.iloc[j,6]) > 6932.5):
            for k in range(len(starlink_dataframe)):
                if((j + k) >= len(starlink_dataframe)):break
                if(float(starlink_dataframe.iloc[j + k,6]) < 6932.5):break
            if(k > 5):
                if(starlink_dataframe.iloc[0,0] not in task_maneuver):
                    task_maneuver.append(starlink_dataframe.iloc[0,0])

In [ ]:
def exchange_date_str(stime):
    stime_list = stime.split(' ')
    if(stime_list[1] == 'Jan'):
        stime_list[1] = stime_list[1].replace('Jan','-01-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Feb'):
        stime_list[1] = stime_list[1].replace('Feb','-02-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Mar'):
        stime_list[1] = stime_list[1].replace('Mar','-03-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Apr'):
        stime_list[1] = stime_list[1].replace('Apr','-04-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'May'):
        stime_list[1] = stime_list[1].replace('May','-05-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Jun'):
        stime_list[1] = stime_list[1].replace('Jun','-06-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Jul'):
        stime_list[1] = stime_list[1].replace('Jul','-07-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Aug'):
        stime_list[1] = stime_list[1].replace('Aug','-08-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Sep'):
        stime_list[1] = stime_list[1].replace('Sep','-09-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Oct'):
        stime_list[1] = stime_list[1].replace('Oct','-10-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Nov'):
        stime_list[1] = stime_list[1].replace('Nov','-11-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    if(stime_list[1] == 'Dec'):
        stime_list[1] = stime_list[1].replace('Dec','-12-')
        stime = "%s%s%s %s"%(stime_list[0],stime_list[1],stime_list[2],stime_list[3])
    return stime
def exchange_crawler_str(ctime):
    ctime = "%s%s"%(ctime[20:24],ctime[3:19])
    ctime = exchange_date_str(ctime)
    return ctime

In [ ]:
#第四步
#这段代码是获取conjunction文件中的数据并存到cunjunction列表中
#先是获取文件夹下所有的文件名
def get_allfile(path):  # 获取所有文件
    all_file = []
    for f in os.listdir(path):  #listdir返回文件中所有目录
        f_name = os.path.join(path, f)
        all_file.append(f_name)
    return all_file
    
tickets = r"C:\Users\zhaow21\Seafile\debris\data\raw_conjunction_data"
all_file=get_allfile(tickets)  #tickets要获取文件夹名
print(all_file[1])

#再根据文件名提取数据
conjunction = {}
conjunction_dataframe = []
for i in range(len(all_file)):#这里的i就是取出文件夹里的第i+1个文件
    xls_file = xlrd.open_workbook(all_file[i])
    xls_sheet = xls_file.sheets()[0]
    nrows = xls_sheet.nrows
    for j in range(1,nrows):
        row_value = xls_sheet.row_values(j)
        #row_value是想要从conjuction中获取的数据字段，按需增删
        #row_value = [row_value[0],row_value[3],row_value[6],row_value[7],row_value[8],row_value[9],row_value[10],row_value[11],row_value[12]]
        #目前这个row_value提取的字段如下
        #max prob  relative volicity  name1  name2  ID  TCA  start time  stop time  time(crawler)
        if(float(row_value[0]) < 1e-5):continue
        sat_id1 = int(row_value[8].split('-')[0])
        sat_id2 = int(row_value[8].split('-')[1])
        tca = exchange_date_str(row_value[9])
        tca = datetime.datetime.strptime(tca,'%Y-%m-%d %H:%M:%S.%f')
        if(sat_id1 not in conjunction.keys()):
            conjunction[sat_id1] = [tca]
        else:
            conjunction[sat_id1].append(tca)
        if(sat_id2 not in conjunction.keys()):
            conjunction[sat_id2] = [tca]
        else:
            conjunction[sat_id2].append(tca)
        row_value[9] = exchange_date_str(row_value[9])
        row_value[9] = datetime.datetime.strptime(row_value[9],'%Y-%m-%d %H:%M:%S.%f')
        #row_value是想要从conjuction中获取的数据字段，按需增删
        row_value = [float(row_value[0]),float(row_value[3]),row_value[6],row_value[7],sat_id1,sat_id2,row_value[9],row_value[10],row_value[11],row_value[12]]
        #目前这个row_value提取的字段如下
        #max prob  relative volicity  name1  name2  ID  TCA  start time  stop time  time(crawler)
        conjunction_dataframe.append(row_value)

C:\Users\zhaow21\Seafile\debris\data\raw_conjunction_data\conjunction_result_100000_MAXPROB_1650556802.0746994.xls


In [ ]:
from datetime import timedelta


result = {}
for key in maneuver.keys():
    if(maneuver[key] == {}):continue
    if(key not in conjunction.keys()):continue
    tca_list = list(maneuver[key].keys())
    for i in range(len(tca_list)):
        for j in range(len(conjunction[key])):
            if(tca_list[i] <= conjunction[key][j]):
                delta_time = conjunction[key][j] - tca_list[i]
                if(timedelta(hours= 11) <= delta_time <= timedelta(hours= 13)):
                    if(key not in result.keys()):
                        result[key] = [conjunction[key][j]]
                    else:
                        result[key].append(conjunction[key][j])

In [ ]:
ca_maneuver_list = []
for key in result.keys():
    if(key not in task_maneuver):ca_maneuver_list.append(key)
len(ca_maneuver_list)

35

In [ ]:
for key in ca_maneuver_list:
    starlink_dict[key]['alltitude'] = ''
    for i in range(len(starlink_dict[key])):#len(starlink_dict[key])
        tle_line1 = starlink_dict[key].iloc[i,4]
        tle_line2 = starlink_dict[key].iloc[i,5]
        #starlink_dict[key].iloc[i,2] = datetime.datetime.strptime(starlink_dict[key].iloc[i,2],"%Y-%m-%d %H:%M:%S")
        tle_epoch = starlink_dict[key].iloc[i,2]
        year = tle_epoch.year
        month = tle_epoch.month
        day = tle_epoch.day
        hour = tle_epoch.hour
        minute = tle_epoch.minute
        second = tle_epoch.second

        ts = load.timescale()
        sat = EarthSatellite(tle_line1,tle_line2,None,ts)
        t = ts.utc(year,month,day,hour,minute,second)
        geocentric = sat.at(t)
        h = w84.height_of(geocentric).km
        starlink_dict[key].iloc[i,7] = h

In [ ]:
ca_maneuver_time = []
ca_maneuver_alt = []
for item in ca_maneuver_list:
    ca_df = starlink_dict[item]
    for i in range(len(start_end[item])):
        start_time = start_end[item][i][0]
        stop_time = start_end[item][i][1]
        mid_df = ca_df[(ca_df['EPOCH'] >= start_time) & (ca_df['EPOCH'] <= stop_time)]
        max_alt = max(mid_df['alltitude'])
        min_alt = min(mid_df['alltitude'])
        ca_maneuver_alt.append(max_alt - min_alt)
        ca_maneuver_time.append(((stop_time - start_time).total_seconds()) / (60 * 60))

## Start from this line

In [ ]:
ca_maneuver_alt = np.load('./data/ca_maneuver_alt.npy',allow_pickle=True).tolist()
ca_maneuver_time = np.load('./data/ca_maneuver_time.npy',allow_pickle=True).tolist()

In [ ]:
max(ca_maneuver_alt)

21.199575476011887

In [ ]:
ca_time = pd.Series(ca_maneuver_time)
ca_time.describe()

count     34.000000
mean     134.141621
std       71.032626
min       12.739208
25%       92.747580
50%      120.508376
75%      181.625326
max      369.420163
dtype: float64

In [ ]:
ca_alt = pd.Series(ca_maneuver_alt)
ca_alt.describe()

count    34.000000
mean      6.183981
std       4.754447
min       1.498825
25%       3.051843
50%       4.998075
75%       6.618164
max      21.199575
dtype: float64